# Load Packages

In [1]:
import functions as f
import pandas as pd

# Load Data

In [2]:
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()

# Run Double ML

In [6]:
first_stage_1, first_stage_2, double_mls = f.global_run(df, splits=3, cols_to_drop_manual=[], iters=50, log=False, intermediary_scores=False)

Running Split 1...
100%|██████████| 50/50 [07:50<00:00,  9.42s/trial, best loss: 0.07594240424345441]
Done!!
Running Split 2...
100%|██████████| 50/50 [06:48<00:00,  8.18s/trial, best loss: 0.03716616162308659]
Done!!
Running Split 3...
100%|██████████| 50/50 [07:31<00:00,  9.04s/trial, best loss: 0.02652627326298899] 
Done!!


In [9]:
for k, v in double_mls.items():
    print(k)
    display(v.summary)
    print(v.sensitivity_analysis().sensitivity_summary)

(0.00852, 0.162]


,coef,std err,t,P>|t|,2.5 %,97.5 %
welcome_discount,0.081339,0.012822,6.343846,2.240994e-10,0.056209,0.106469


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
                  CI lower  theta lower     theta  theta upper  CI upper
welcome_discount  0.000871     0.021817  0.081339     0.140861  0.162331

------------------ Robustness Values ------------------
                  H_0    RV (%)   RVa (%)
welcome_discount  0.0  4.076821  3.043358
(0.162, 0.243]


,coef,std err,t,P>|t|,2.5 %,97.5 %
welcome_discount,0.117591,0.008372,14.045109,8.254120e-45,0.101182,0.134001


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
                  CI lower  theta lower     theta  theta upper  CI upper
welcome_discount  0.010557     0.024573  0.117591     0.210609  0.224357

------------------ Robustness Values ------------------
                  H_0    RV (%)   RVa (%)
welcome_discount  0.0  3.777355  3.333491
(0.243, 0.3]


,coef,std err,t,P>|t|,2.5 %,97.5 %
welcome_discount,0.20539,0.010275,19.989464,6.802218e-89,0.185251,0.225528


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
                  CI lower  theta lower    theta  theta upper  CI upper
welcome_discount  0.079594     0.096812  0.20539     0.313968   0.33075

------------------ Robustness Values ------------------
                  H_0    RV (%)   RVa (%)
welcome_discount  0.0  5.598397  5.132146
